In [1]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

meta_ds = GLUEMetaDataset(k=4,numTasks=1000)

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 630.53it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 113.05it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|███████████████

In [2]:
train_protomaml_sampler = FewShotEpisodeBatchSampler(meta_ds, kShot=4, batchSize=4)

train_protomaml_loader = data.DataLoader(
    meta_ds, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=4)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [4]:
# from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

In [5]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=4)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

for i in range(1):
    batch = next(iter(val_protomaml_loader))
    for episode_i in range(len(batch[0])):
        data, labels = batch[0][episode_i], batch[1][episode_i]
        supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)
#         print(labels)

In [6]:
from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.validation_step(next(iter(val_protomaml_loader)), 0)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from training.trainer import train_model

protomaml_model = train_model(
    ProtoFOMAML,
    train_loader=train_protomaml_loader,
    val_loader=val_protomaml_loader,
    outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=10, batchSize=8, warmupSteps=0
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name        | Type               | Params
---------------------------------------------------
0 | metaLearner | PrototypeMetaModel | 108 M 
---------------------------------------------------
14.4 M    Trainable params
94.1 M    Non-trainable params
108 M     Total params
434.029   Total estimated model params size (MB)


Training ProtoFOMAML with parameters "batchSize":   8
"innerLR":     0.001
"outerLR":     0.0005
"outputLR":    0.01
"steps":       10
"warmupSteps": 0
Number of labels in the episode are 2 and lines are 1
inner loop training loss is 4.673847675323486 and accuracy is 1.0
inner loop training loss is 2.0927298069000244 and accuracy is 1.0
inner loop training loss is 2.828368663787842 and accuracy is 1.0
inner loop training loss is 0.1372455209493637 and accuracy is 1.0
inner loop training loss is 0.23103585839271545 and accuracy is 1.0
inner loop training loss is 0.0012517105787992477 and accuracy is 1.0
inner loop training loss is 1.68084297911264e-05 and accuracy is 1.0
inner loop training loss is 2.3841855067985307e-07 and accuracy is 1.0
inner loop training loss is 0 and accuracy is 1.0


/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('inner_loop_training_loss', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


inner loop training loss is 0 and accuracy is 1.0
outer loop training accuracy is 0.625 and loss is 48.93934631347656
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 24.792964935302734 and accuracy is 0.5833333333333334
inner loop training loss is 16.948206901550293 and accuracy is 0.4166666666666667
inner loop training loss is 14.454851627349854 and accuracy is 0.5
inner loop training loss is 14.992933869361877 and accuracy is 0.6666666666666666
inner loop training loss is 18.586313143372536 and accuracy is 0.6666666666666666
inner loop training loss is 10.356051921844482 and accuracy is 0.6666666666666666
inner loop training loss is 7.925846420228481 and accuracy is 0.6666666666666666
inner loop training loss is 7.378769635812205 and accuracy is 0.8333333333333334
inner loop training loss is 3.638570427894585 and accuracy is 0.75
inner loop training loss is 1.2384074926376272 and accuracy is 1.0
outer loop training accuracy is 0.16666666666666666 and

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/loops/optimization/automatic.py:129: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Number of labels in the episode are 2 and lines are 1
inner loop training loss is 5.012779712677002 and accuracy is 0.625
inner loop training loss is 5.97076940536499 and accuracy is 0.5
inner loop training loss is 16.657569885253906 and accuracy is 0.5
inner loop training loss is 12.41330337524414 and accuracy is 0.5
inner loop training loss is 5.495307445526123 and accuracy is 0.5
inner loop training loss is 6.253486156463623 and accuracy is 0.5
inner loop training loss is 7.9357709884643555 and accuracy is 0.5
inner loop training loss is 12.170990943908691 and accuracy is 0.625
inner loop training loss is 8.039926528930664 and accuracy is 0.5
inner loop training loss is 4.899036884307861 and accuracy is 0.75
outer loop training accuracy is 0.5 and loss is 10.931307792663574
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 24.708322525024414 and accuracy is 0.3333333333333333
inner loop training loss is 20.747663497924805 and accuracy is 0.41666666666

Number of labels in the episode are 2 and lines are 1
inner loop training loss is 5.296738624572754 and accuracy is 0.625
inner loop training loss is 2.566908836364746 and accuracy is 0.875
inner loop training loss is 11.850149154663086 and accuracy is 0.5
inner loop training loss is 4.286468029022217 and accuracy is 0.5
inner loop training loss is 1.593632459640503 and accuracy is 0.875
inner loop training loss is 0.32952407002449036 and accuracy is 1.0
inner loop training loss is 0.04608083888888359 and accuracy is 1.0
inner loop training loss is 0.0010287227341905236 and accuracy is 1.0
inner loop training loss is 7.152556236178498e-07 and accuracy is 1.0
inner loop training loss is 0 and accuracy is 1.0
outer loop training accuracy is 0.5 and loss is 46.978233337402344
Number of labels in the episode are 2 and lines are 1
inner loop training loss is 4.975506782531738 and accuracy is 0.75
inner loop training loss is 2.1454739570617676 and accuracy is 1.0
inner loop training loss is 

In [ ]:
import os
import torch

file_path = "/home/aksingh/Downloads/version_6/checkpoints/"

for file in os.listdir(file_path):
    if file.endswith(".ckpt"):
        model_path = os.path.join(file_path, file)

model = ProtoFOMAML.load_from_checkpoint(model_path, map_location=torch.device('cpu'))

In [ ]:
print(model.metaLearner)